<h2>Auswahl des Basismodells</h2>

Es werden verschiedene auf SQuaD trainierte BERT-varianten auf dem Validationsplit des Datensatzes getestet, um das am besten für den Datensatz geeignete zu finden; Dabei werden die Reader isoliert evaliert, um einzuschätzen, wieviel sie zur Pipeline beitragen

Die Modelle, die im folgenden evaluiert werden, sind:

- roBERTA base finetuned on SQuaD v2:

     > deepset/roberta-base-squad2
    
- roBERTA base finetuned on SQuaD v2 distilled: 

     > deepset/roberta-base-squad2-distilled
    
- BERT base cased finetuned on SQuaD v2:

     > deepset/bert-base-cased-squad2 

- distilBERT base cased finetuned on SQuaD v1.1:

     > distilbert-base-cased-distilled-squad 

- electra base cased finetuned on SQuaD v2:

     > deepset/electra-base-squad2
    
    
- alBERT base v2 finetuned on SQuaD v2:

     > twmkn9/albert-base-v2-squad2
    
- deBERTaV3 base finetuned on SQuaD v2:

     > deepset/deberta-v3-base-squad2

In [1]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.1.2
    Uninstalling pip-22.1.2:
      Successfully uninstalled pip-22.1.2
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-xkxrz4s3/farm-haystack_7c068e3b500f4055afb7d8f2fa63e37f
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-xkxrz4s3/farm-haystack_7c068e3b500f4055afb7d8f2fa63e37f
  Resolved https://github.com/deepset-ai/haystack.git to commit 820742cac7c6d8426bb9d465106193cf79944686
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 8.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━

Laden Des Validationsplits:

In [2]:
!unzip "./val_split_squad_format.json.zip"

Archive:  ./val_split_squad_format.json.zip
  inflating: val_split_squad_format.json  


<h3>roBERTA base finetuned on SQuaD v2:</h3>




In [3]:
from haystack.nodes import FARMReader

reader1 = FARMReader("deepset/roberta-base-squad2", top_k=10, return_no_answer=True, use_gpu=True)


In [4]:
# Evaluate Reader on its own
reader_eval_results = reader1.eval_on_file("./","./val_split_squad_format.json")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.
Evaluating: 100%|██████████| 6957/6957 [1:31:32<00:00,  1.27it/s]
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
top_n = reader_eval_results["top_n"]

# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer including no_answers
print(f"Reader Top-{top_n}-Accuracy:", reader_eval_results["top_n_accuracy"])
# Reader Top-1-Exact Match is the proportion of questions where the first predicted answer is exactly the same as the correct answer including no_answers
print("Reader Top-1-Exact Match:", reader_eval_results["EM"])
# Reader Top-1-F1-Score is the average overlap between the first predicted answers and the correct answers including no_answers
print("Reader Top-1-F1-Score:", reader_eval_results["f1"])
# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer excluding no_answers
print(f"Reader Top-{top_n}-Accuracy (without no_answers):", reader_eval_results["top_n_accuracy_text_answer"])
# Reader Top-N-Exact Match is the proportion of questions where the predicted answer within the first n results is exactly the same as the correct answer excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-Exact Match (without no_answers):", reader_eval_results["top_n_EM_text_answer"])
# Reader Top-N-F1-Score is the average overlap between the top n predicted answers and the correct answers excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-F1-Score (without no_answers):", reader_eval_results["top_n_f1_text_answer"])

Reader Top-4-Accuracy: 93.2258064516129
Reader Top-1-Exact Match: 0.3225806451612903
Reader Top-1-F1-Score: 19.990153306570217
Reader Top-4-Accuracy (without no_answers): 93.2258064516129
Reader Top-4-Exact Match (without no_answers): 1.0752688172043012
Reader Top-4-F1-Score (without no_answers): 53.62720823220732


In [7]:
import json

with open('./roberta_base_squad2_evaldata.json', 'w') as f:
    json.dump(reader_eval_results, f)

In [12]:
del reader1, reader_eval_results, top_n, f

<h3>roBERTA base finetuned on SQuaD v2 distilled:</h3>

In [8]:
reader2 = FARMReader("deepset/roberta-base-squad2-distilled", top_k=10, return_no_answer=True, use_gpu=True)


In [9]:
# Evaluate Reader on its own
reader_eval_results2 = reader2.eval_on_file("./","./val_split_squad_format.json")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.
Evaluating: 100%|██████████| 6957/6957 [1:31:26<00:00,  1.27it/s]
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [14]:
top_n = reader_eval_results2["top_n"]

# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer including no_answers
print(f"Reader Top-{top_n}-Accuracy:", reader_eval_results2["top_n_accuracy"])
# Reader Top-1-Exact Match is the proportion of questions where the first predicted answer is exactly the same as the correct answer including no_answers
print("Reader Top-1-Exact Match:", reader_eval_results2["EM"])
# Reader Top-1-F1-Score is the average overlap between the first predicted answers and the correct answers including no_answers
print("Reader Top-1-F1-Score:", reader_eval_results2["f1"])
# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer excluding no_answers
print(f"Reader Top-{top_n}-Accuracy (without no_answers):", reader_eval_results2["top_n_accuracy_text_answer"])
# Reader Top-N-Exact Match is the proportion of questions where the predicted answer within the first n results is exactly the same as the correct answer excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-Exact Match (without no_answers):", reader_eval_results2["top_n_EM_text_answer"])
# Reader Top-N-F1-Score is the average overlap between the top n predicted answers and the correct answers excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-F1-Score (without no_answers):", reader_eval_results2["top_n_f1_text_answer"])

Reader Top-4-Accuracy: 92.90322580645162
Reader Top-1-Exact Match: 0.43010752688172044
Reader Top-1-F1-Score: 26.78842267121092
Reader Top-4-Accuracy (without no_answers): 92.90322580645162
Reader Top-4-Exact Match (without no_answers): 1.2903225806451613
Reader Top-4-F1-Score (without no_answers): 58.157160933739526


In [15]:
with open('./roberta_base_squad2_distilled_evaldata.json', 'w') as f:
    json.dump(reader_eval_results2, f)

In [16]:
del reader2, reader_eval_results2, top_n, f

<h3>BERT base cased finetuned on SQuaD v2:</h3> 

In [17]:
reader3 = FARMReader("deepset/bert-base-cased-squad2", top_k=10, return_no_answer=True, use_gpu=True)


In [18]:
# Evaluate Reader on its own
reader_eval_results3 = reader3.eval_on_file("./","./val_split_squad_format.json")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.
Evaluating: 100%|██████████| 7377/7377 [1:37:38<00:00,  1.26it/s]
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [19]:
top_n = reader_eval_results3["top_n"]

# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer including no_answers
print(f"Reader Top-{top_n}-Accuracy:", reader_eval_results3["top_n_accuracy"])
# Reader Top-1-Exact Match is the proportion of questions where the first predicted answer is exactly the same as the correct answer including no_answers
print("Reader Top-1-Exact Match:", reader_eval_results3["EM"])
# Reader Top-1-F1-Score is the average overlap between the first predicted answers and the correct answers including no_answers
print("Reader Top-1-F1-Score:", reader_eval_results3["f1"])
# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer excluding no_answers
print(f"Reader Top-{top_n}-Accuracy (without no_answers):", reader_eval_results3["top_n_accuracy_text_answer"])
# Reader Top-N-Exact Match is the proportion of questions where the predicted answer within the first n results is exactly the same as the correct answer excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-Exact Match (without no_answers):", reader_eval_results3["top_n_EM_text_answer"])
# Reader Top-N-F1-Score is the average overlap between the top n predicted answers and the correct answers excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-F1-Score (without no_answers):", reader_eval_results3["top_n_f1_text_answer"])

Reader Top-4-Accuracy: 93.33333333333333
Reader Top-1-Exact Match: 0.21505376344086022
Reader Top-1-F1-Score: 26.024205229795278
Reader Top-4-Accuracy (without no_answers): 93.33333333333333
Reader Top-4-Exact Match (without no_answers): 1.935483870967742
Reader Top-4-F1-Score (without no_answers): 60.70766441286568


In [20]:
with open('./bert_base_cased_squad2_evaldata.json', 'w') as f:
    json.dump(reader_eval_results3, f)

In [21]:
del reader3, reader_eval_results3, top_n, f

<h3>distilBERT base cased finetuned on SQuaD v1.1:</h3>

In [22]:
reader4 = FARMReader("distilbert-base-cased-distilled-squad", top_k=10, return_no_answer=True, use_gpu=True)


In [23]:
# Evaluate Reader on its own
reader_eval_results4 = reader4.eval_on_file("./","./val_split_squad_format.json")

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Evaluating: 100%|██████████| 7377/7377 [50:07<00:00,  2.45it/s]


In [24]:
top_n = reader_eval_results4["top_n"]

# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer including no_answers
print(f"Reader Top-{top_n}-Accuracy:", reader_eval_results4["top_n_accuracy"])
# Reader Top-1-Exact Match is the proportion of questions where the first predicted answer is exactly the same as the correct answer including no_answers
print("Reader Top-1-Exact Match:", reader_eval_results4["EM"])
# Reader Top-1-F1-Score is the average overlap between the first predicted answers and the correct answers including no_answers
print("Reader Top-1-F1-Score:", reader_eval_results4["f1"])
# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer excluding no_answers
print(f"Reader Top-{top_n}-Accuracy (without no_answers):", reader_eval_results4["top_n_accuracy_text_answer"])
# Reader Top-N-Exact Match is the proportion of questions where the predicted answer within the first n results is exactly the same as the correct answer excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-Exact Match (without no_answers):", reader_eval_results4["top_n_EM_text_answer"])
# Reader Top-N-F1-Score is the average overlap between the top n predicted answers and the correct answers excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-F1-Score (without no_answers):", reader_eval_results4["top_n_f1_text_answer"])

Reader Top-4-Accuracy: 95.59139784946237
Reader Top-1-Exact Match: 0.43010752688172044
Reader Top-1-F1-Score: 28.08505179326812
Reader Top-4-Accuracy (without no_answers): 95.59139784946237
Reader Top-4-Exact Match (without no_answers): 0.8602150537634409
Reader Top-4-F1-Score (without no_answers): 54.97198788796345


In [25]:
with open('./distilbert_base_cased_distilled_squad_evaldata.json', 'w') as f:
    json.dump(reader_eval_results4, f)

In [26]:
del reader4, reader_eval_results4, top_n, f

<h3>Electra base finetuned on SQuaD v2:</h3>

In [4]:
reader5 = FARMReader("deepset/electra-base-squad2", top_k=10, return_no_answer=True, use_gpu=True)


In [5]:
# Evaluate Reader on its own
reader_eval_results5 = reader5.eval_on_file("./","./val_split_squad_format.json")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.
Evaluating: 100%|██████████| 7145/7145 [34:31<00:00,  3.45it/s]
/opt/conda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [6]:
top_n = reader_eval_results5["top_n"]

# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer including no_answers
print(f"Reader Top-{top_n}-Accuracy:", reader_eval_results5["top_n_accuracy"])
# Reader Top-1-Exact Match is the proportion of questions where the first predicted answer is exactly the same as the correct answer including no_answers
print("Reader Top-1-Exact Match:", reader_eval_results5["EM"])
# Reader Top-1-F1-Score is the average overlap between the first predicted answers and the correct answers including no_answers
print("Reader Top-1-F1-Score:", reader_eval_results5["f1"])
# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer excluding no_answers
print(f"Reader Top-{top_n}-Accuracy (without no_answers):", reader_eval_results5["top_n_accuracy_text_answer"])
# Reader Top-N-Exact Match is the proportion of questions where the predicted answer within the first n results is exactly the same as the correct answer excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-Exact Match (without no_answers):", reader_eval_results5["top_n_EM_text_answer"])
# Reader Top-N-F1-Score is the average overlap between the top n predicted answers and the correct answers excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-F1-Score (without no_answers):", reader_eval_results5["top_n_f1_text_answer"])

Reader Top-4-Accuracy: 93.87096774193549
Reader Top-1-Exact Match: 0.0
Reader Top-1-F1-Score: 29.4365829685648
Reader Top-4-Accuracy (without no_answers): 93.87096774193549
Reader Top-4-Exact Match (without no_answers): 1.6129032258064515
Reader Top-4-F1-Score (without no_answers): 59.574073137399886


In [9]:
with open('./electra_base_squad2_evaldata.json', 'w') as f:
    json.dump(reader_eval_results5, f)

In [10]:
del reader5, reader_eval_results5, top_n, f

<h3>alBERT base v2 finetuned on SQuaD v2:</h3>

In [11]:
reader6 = FARMReader("twmkn9/albert-base-v2-squad2", top_k=10, return_no_answer=True, use_gpu=True)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [12]:
# Evaluate Reader on its own
reader_eval_results6 = reader6.eval_on_file("./","./val_split_squad_format.json")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.
Evaluating: 100%|██████████| 7263/7263 [41:17<00:00,  2.93it/s]


In [13]:
top_n = reader_eval_results6["top_n"]

# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer including no_answers
print(f"Reader Top-{top_n}-Accuracy:", reader_eval_results6["top_n_accuracy"])
# Reader Top-1-Exact Match is the proportion of questions where the first predicted answer is exactly the same as the correct answer including no_answers
print("Reader Top-1-Exact Match:", reader_eval_results6["EM"])
# Reader Top-1-F1-Score is the average overlap between the first predicted answers and the correct answers including no_answers
print("Reader Top-1-F1-Score:", reader_eval_results6["f1"])
# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer excluding no_answers
print(f"Reader Top-{top_n}-Accuracy (without no_answers):", reader_eval_results6["top_n_accuracy_text_answer"])
# Reader Top-N-Exact Match is the proportion of questions where the predicted answer within the first n results is exactly the same as the correct answer excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-Exact Match (without no_answers):", reader_eval_results6["top_n_EM_text_answer"])
# Reader Top-N-F1-Score is the average overlap between the top n predicted answers and the correct answers excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-F1-Score (without no_answers):", reader_eval_results6["top_n_f1_text_answer"])

Reader Top-4-Accuracy: 93.76344086021506
Reader Top-1-Exact Match: 0.3225806451612903
Reader Top-1-F1-Score: 22.857484082896086
Reader Top-4-Accuracy (without no_answers): 93.76344086021506
Reader Top-4-Exact Match (without no_answers): 1.6129032258064515
Reader Top-4-F1-Score (without no_answers): 55.91406956181508


In [14]:
with open('./albert_base_v2_squad2_evaldata.json', 'w') as f:
    json.dump(reader_eval_results6, f)

In [15]:
del reader6, reader_eval_results6, top_n, f

<h3>deBERTaV3 base finetuned on SQuaD v2:</h3>

In [16]:
reader7 = FARMReader("deepset/deberta-v3-base-squad2", top_k=10, return_no_answer=True, use_gpu=True)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [17]:
# Evaluate Reader on its own
reader_eval_results7 = reader7.eval_on_file("./","./val_split_squad_format.json")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.
Evaluating: 100%|██████████| 6914/6914 [45:57<00:00,  2.51it/s]


In [18]:
top_n = reader_eval_results7["top_n"]

# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer including no_answers
print(f"Reader Top-{top_n}-Accuracy:", reader_eval_results7["top_n_accuracy"])
# Reader Top-1-Exact Match is the proportion of questions where the first predicted answer is exactly the same as the correct answer including no_answers
print("Reader Top-1-Exact Match:", reader_eval_results7["EM"])
# Reader Top-1-F1-Score is the average overlap between the first predicted answers and the correct answers including no_answers
print("Reader Top-1-F1-Score:", reader_eval_results7["f1"])
# Reader Top-N-Accuracy is the proportion of predicted answers that match with their corresponding correct answer excluding no_answers
print(f"Reader Top-{top_n}-Accuracy (without no_answers):", reader_eval_results7["top_n_accuracy_text_answer"])
# Reader Top-N-Exact Match is the proportion of questions where the predicted answer within the first n results is exactly the same as the correct answer excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-Exact Match (without no_answers):", reader_eval_results7["top_n_EM_text_answer"])
# Reader Top-N-F1-Score is the average overlap between the top n predicted answers and the correct answers excluding no_answers (no_answers are always present within top n).
print(f"Reader Top-{top_n}-F1-Score (without no_answers):", reader_eval_results7["top_n_f1_text_answer"])

Reader Top-4-Accuracy: 93.87096774193549
Reader Top-1-Exact Match: 0.5376344086021506
Reader Top-1-F1-Score: 30.065917093158813
Reader Top-4-Accuracy (without no_answers): 93.87096774193549
Reader Top-4-Exact Match (without no_answers): 2.903225806451613
Reader Top-4-F1-Score (without no_answers): 62.09464491912499


In [19]:
with open('deberta_v3_base_squad2_evaldata.json', 'w') as f:
    json.dump(reader_eval_results7, f)

In [20]:
del reader7, reader_eval_results7, top_n, f